<a href="https://colab.research.google.com/github/chawlaKat/Notebook-Noodlings/blob/master/Get_Scattered_Fancy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[insert intro here]

get_scattered got too long

im trying to make a decent-looking interactive scatter plot from .arff data

In [2]:
pip install liac_arff

In [0]:
import bark_to_byte as bb
import altair as alt
import pandas as pd

In [0]:
algSource = bb.arff_to_dataframe('abstracted_algorithm_runs.arff')

In [0]:
alg_rekeyed = algSource.set_index('instance_id')
alg_joined = alg_rekeyed.join(alg_rekeyed, lsuffix='_x', rsuffix='_y')
alg_joined.reset_index(inplace=True)

In [0]:
four_colors = ['#ffe200', '#f5ffff', '#6600db', '#11182b']
three_colors = ['#dd1f75', '#fda900', '#2a4dac']
three_colors_alt = ['#6754ad', '#ffffff', '#2d5d00']

In [0]:
def make_graph(data_source, x_axis, y_axis, point_marker = 'circle'):
  
  if(point_marker == 'circle'):
      new_graph = alt.Chart(data_source).mark_circle(size=60).encode(
          x = x_axis,
          y = y_axis
      )
      
  elif(point_marker == 'rect'):
      new_graph = alt.Chart(data_source).mark_rect().encode(
          x = x_axis,
          y = y_axis
      )
      
  elif(point_marker == 'bar'):
      new_graph = alt.Chart(data_source).mark_bar().encode(
          x = x_axis,
          y = y_axis
      )
    
  else:
      new_graph = alt.Chart(data_source).mark_point().encode(
          x = x_axis,
          y = y_axis
      )
  
  return new_graph

In [0]:
def make_dropdown(data_source, column, name_prefix):
  opts = list(data_source[column].unique())
  dropdown = alt.binding_select(options = opts)
  select = alt.selection_single(fields = [column],
                                bind = dropdown,
                                name = name_prefix
                               )
  
  return select

In [0]:
def make_selection(array_of_fields, is_multi = False):
  if (is_multi):
    selection = alt.selection_multi(fields = array_of_fields)
  else:
    selection = alt.selection_single(fields = array_of_fields)
    
  return selection

In [0]:
def make_color_scheme(data_source, column, colors, data_type_encoding = 'N'):
  scheme_domain = list(data_source[column].unique())
  scheme_range = colors
  
  scheme_scale = alt.Scale(domain = scheme_domain,
                    range = scheme_range)
  
  #altair needs to know if data is nominal, quantitative, ordinal, or temporal
  if(data_type_encoding == 'N' or data_type_encoding == 'Q' or
      data_type_encoding == 'O' or data_type_encoding == 'T'):
    column_typed = column + ':' + data_type_encoding
  else:
    column_typed = column + ':N'
  
  color_scheme = alt.Color(column_typed,
                           legend = None,
                           scale = scheme_scale)
  
  return color_scheme

In [0]:
def make_color_condition(selector, color_true, color_false = alt.value('darkgray')):
  condition = alt.condition(selector,
                            color_true,
                            color_false)
  
  return condition

In [0]:
def apply_selection_color(chart, selector, color_condition):
  colored_chart = chart.add_selection(
        selector
      ).encode(
        color = color_condition
      )
  
  return colored_chart

In [0]:
def create_comparative_scatter(data_source, key, x_category, y_category, 
                               x_data, y_data, x_filter, y_filter,
                               data_color):
  
  #filter data on x and y axis
  data_legend_base = make_graph(data_source, x_category, y_category, 'rect')
  data_legend_selector = make_selection([x_category, y_category])
  data_legend_colors = make_color_scheme(data_source, y_category, data_color)
  data_legend_update = make_color_condition(data_legend_selector, 
                                            data_legend_colors)
  data_legend = apply_selection_color(data_legend_base, data_legend_selector,
                                      data_legend_update)
  
  #filter data based on attribute
  filter_legend_base = make_graph(data_source, x_filter, y_filter, 'rect')
  filter_legend_selector = make_selection([x_filter, y_filter], True)
  filter_legend_colors = make_color_scheme(data_source, y_filter, data_color)
  filter_legend_update = make_color_condition(filter_legend_selector,
                                              filter_legend_colors)
  filter_legend = apply_selection_color(filter_legend_base, 
                                        filter_legend_selector, 
                                        filter_legend_update)
  
  #set up scatter plot to listen to both legends
  scatter_base = make_graph(data_source, x_data, y_data)
  scatter_tooltip = scatter_base.encode(
                                tooltip = [key, x_category, x_data, x_filter,
                                           y_category, y_data, y_filter]
                            )
  scatter_plot = scatter_tooltip.transform_filter(data_legend_selector
                            ).transform_filter(filter_legend_selector
                            ).interactive()
  
  
  comparative_chart = (data_legend & filter_legend) | scatter_plot
  
  return comparative_chart

In [0]:
algChart = make_graph(algSource, 'algorithm', 'sum(runtime)', 'bar')

algChart

Chart({
  data:    instance_id  repetition       algorithm   runtime runstatus
  0   Instance_0         1.0     algorithm_0  3600.000    memout
  1   Instance_0         1.0     algorithm_1  3600.000    memout
  2   Instance_0         1.0  id_algorithm_0   311.863        ok
  3   Instance_0         1.0  id_algorithm_1  3600.000   timeout
  4   Instance_1         1.0     algorithm_0  3600.000    memout
  5   Instance_1         1.0     algorithm_1  3600.000    memout
  6   Instance_1         1.0  id_algorithm_0    78.121        ok
  7   Instance_1         1.0  id_algorithm_1   600.402        ok
  8   Instance_2         1.0     algorithm_0  3600.000    memout
  9   Instance_2         1.0     algorithm_1  3600.000    memout
  10  Instance_2         1.0  id_algorithm_0     0.012        ok
  11  Instance_2         1.0  id_algorithm_1    60.720        ok
  12  Instance_3         1.0     algorithm_0  3600.000    memout
  13  Instance_3         1.0     algorithm_1    92.986        ok
  14  Instance_3         1.0  id_algorithm_0  2031.039        ok
  15  Instance_3         1.0  id_algorithm_1   141.857        ok
  16  Instance_4         1.0     algorithm_0  3600.000    memout
  17  Instance_4         1.0     algorithm_1  3600.000    memout
  18  Instance_4         1.0  id_algorithm_0   422.530        ok
  19  Instance_4         1.0  id_algorithm_1  3600.000   timeout
  20  Instance_5         1.0     algorithm_0   182.019        ok
  21  Instance_5         1.0     algorithm_1  3600.000    memout
  22  Instance_5         1.0  id_algorithm_0    66.988        ok
  23  Instance_5         1.0  id_algorithm_1   249.488        ok,
  encoding: FacetedEncoding({
    x: X({
      field: 'algorithm',
      type: 'nominal'
    }),
    y: Y({
      aggregate: 'sum',
      field: 'runtime',
      type: 'quantitative'
    })
  }),
  mark: 'bar'
})

In [0]:
instance_dropdown = make_dropdown(algSource, 'instance_id', 'in_id')

algDropdown = algChart.add_selection(
    instance_dropdown).transform_filter(
    instance_dropdown)

algDropdown

Chart({
  data:    instance_id  repetition       algorithm   runtime runstatus
  0   Instance_0         1.0     algorithm_0  3600.000    memout
  1   Instance_0         1.0     algorithm_1  3600.000    memout
  2   Instance_0         1.0  id_algorithm_0   311.863        ok
  3   Instance_0         1.0  id_algorithm_1  3600.000   timeout
  4   Instance_1         1.0     algorithm_0  3600.000    memout
  5   Instance_1         1.0     algorithm_1  3600.000    memout
  6   Instance_1         1.0  id_algorithm_0    78.121        ok
  7   Instance_1         1.0  id_algorithm_1   600.402        ok
  8   Instance_2         1.0     algorithm_0  3600.000    memout
  9   Instance_2         1.0     algorithm_1  3600.000    memout
  10  Instance_2         1.0  id_algorithm_0     0.012        ok
  11  Instance_2         1.0  id_algorithm_1    60.720        ok
  12  Instance_3         1.0     algorithm_0  3600.000    memout
  13  Instance_3         1.0     algorithm_1    92.986        ok
  14  Instance_3         1.0  id_algorithm_0  2031.039        ok
  15  Instance_3         1.0  id_algorithm_1   141.857        ok
  16  Instance_4         1.0     algorithm_0  3600.000    memout
  17  Instance_4         1.0     algorithm_1  3600.000    memout
  18  Instance_4         1.0  id_algorithm_0   422.530        ok
  19  Instance_4         1.0  id_algorithm_1  3600.000   timeout
  20  Instance_5         1.0     algorithm_0   182.019        ok
  21  Instance_5         1.0     algorithm_1  3600.000    memout
  22  Instance_5         1.0  id_algorithm_0    66.988        ok
  23  Instance_5         1.0  id_algorithm_1   249.488        ok,
  encoding: FacetedEncoding({
    x: X({
      field: 'algorithm',
      type: 'nominal'
    }),
    y: Y({
      aggregate: 'sum',
      field: 'runtime',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  selection: {'in_id': SelectionDef({
    bind: BindRadioSelect({
      input: 'select',
      options: ['Instance_0', 'Instance_1', 'Instance_2', 'Instance_3', 'Instance_4', 'Instance_5']
    }),
    fields: ['instance_id'],
    type: 'single'
  })},
  transform: [FilterTransform({
    filter: {'selection': 'in_id'}
  })]
})

In [0]:
alg_legend = make_graph(alg_joined, 'algorithm_x', 'algorithm_y', 'rect')

alg_legend

Chart({
  data:    instance_id  repetition_x  ... runtime_y  runstatus_y
  0   Instance_0           1.0  ...  3600.000       memout
  1   Instance_0           1.0  ...  3600.000       memout
  2   Instance_0           1.0  ...   311.863           ok
  3   Instance_0           1.0  ...  3600.000      timeout
  4   Instance_0           1.0  ...  3600.000       memout
  5   Instance_0           1.0  ...  3600.000       memout
  6   Instance_0           1.0  ...   311.863           ok
  7   Instance_0           1.0  ...  3600.000      timeout
  8   Instance_0           1.0  ...  3600.000       memout
  9   Instance_0           1.0  ...  3600.000       memout
  10  Instance_0           1.0  ...   311.863           ok
  11  Instance_0           1.0  ...  3600.000      timeout
  12  Instance_0           1.0  ...  3600.000       memout
  13  Instance_0           1.0  ...  3600.000       memout
  14  Instance_0           1.0  ...   311.863           ok
  15  Instance_0           1.0  ...  3600.000      timeout
  16  Instance_1           1.0  ...  3600.000       memout
  17  Instance_1           1.0  ...  3600.000       memout
  18  Instance_1           1.0  ...    78.121           ok
  19  Instance_1           1.0  ...   600.402           ok
  20  Instance_1           1.0  ...  3600.000       memout
  21  Instance_1           1.0  ...  3600.000       memout
  22  Instance_1           1.0  ...    78.121           ok
  23  Instance_1           1.0  ...   600.402           ok
  24  Instance_1           1.0  ...  3600.000       memout
  25  Instance_1           1.0  ...  3600.000       memout
  26  Instance_1           1.0  ...    78.121           ok
  27  Instance_1           1.0  ...   600.402           ok
  28  Instance_1           1.0  ...  3600.000       memout
  29  Instance_1           1.0  ...  3600.000       memout
  ..         ...           ...  ...       ...          ...
  66  Instance_4           1.0  ...   422.530           ok
  67  Instance_4           1.0  ...  3600.000      timeout
  68  Instance_4           1.0  ...  3600.000       memout
  69  Instance_4           1.0  ...  3600.000       memout
  70  Instance_4           1.0  ...   422.530           ok
  71  Instance_4           1.0  ...  3600.000      timeout
  72  Instance_4           1.0  ...  3600.000       memout
  73  Instance_4           1.0  ...  3600.000       memout
  74  Instance_4           1.0  ...   422.530           ok
  75  Instance_4           1.0  ...  3600.000      timeout
  76  Instance_4           1.0  ...  3600.000       memout
  77  Instance_4           1.0  ...  3600.000       memout
  78  Instance_4           1.0  ...   422.530           ok
  79  Instance_4           1.0  ...  3600.000      timeout
  80  Instance_5           1.0  ...   182.019           ok
  81  Instance_5           1.0  ...  3600.000       memout
  82  Instance_5           1.0  ...    66.988           ok
  83  Instance_5           1.0  ...   249.488           ok
  84  Instance_5           1.0  ...   182.019           ok
  85  Instance_5           1.0  ...  3600.000       memout
  86  Instance_5           1.0  ...    66.988           ok
  87  Instance_5           1.0  ...   249.488           ok
  88  Instance_5           1.0  ...   182.019           ok
  89  Instance_5           1.0  ...  3600.000       memout
  90  Instance_5           1.0  ...    66.988           ok
  91  Instance_5           1.0  ...   249.488           ok
  92  Instance_5           1.0  ...   182.019           ok
  93  Instance_5           1.0  ...  3600.000       memout
  94  Instance_5           1.0  ...    66.988           ok
  95  Instance_5           1.0  ...   249.488           ok
  
  [96 rows x 9 columns],
  encoding: FacetedEncoding({
    x: X({
      field: 'algorithm_x',
      type: 'nominal'
    }),
    y: Y({
      field: 'algorithm_y',
      type: 'nominal'
    })
  }),
  mark: 'rect'
})

In [0]:
alg_colors = make_color_scheme(alg_joined, 'algorithm_y', four_colors)

alg_legend_colored = alg_legend.encode(
    color = alg_colors
)

alg_legend_colored

Chart({
  data:    instance_id  repetition_x  ... runtime_y  runstatus_y
  0   Instance_0           1.0  ...  3600.000       memout
  1   Instance_0           1.0  ...  3600.000       memout
  2   Instance_0           1.0  ...   311.863           ok
  3   Instance_0           1.0  ...  3600.000      timeout
  4   Instance_0           1.0  ...  3600.000       memout
  5   Instance_0           1.0  ...  3600.000       memout
  6   Instance_0           1.0  ...   311.863           ok
  7   Instance_0           1.0  ...  3600.000      timeout
  8   Instance_0           1.0  ...  3600.000       memout
  9   Instance_0           1.0  ...  3600.000       memout
  10  Instance_0           1.0  ...   311.863           ok
  11  Instance_0           1.0  ...  3600.000      timeout
  12  Instance_0           1.0  ...  3600.000       memout
  13  Instance_0           1.0  ...  3600.000       memout
  14  Instance_0           1.0  ...   311.863           ok
  15  Instance_0           1.0  ...  3600.000      timeout
  16  Instance_1           1.0  ...  3600.000       memout
  17  Instance_1           1.0  ...  3600.000       memout
  18  Instance_1           1.0  ...    78.121           ok
  19  Instance_1           1.0  ...   600.402           ok
  20  Instance_1           1.0  ...  3600.000       memout
  21  Instance_1           1.0  ...  3600.000       memout
  22  Instance_1           1.0  ...    78.121           ok
  23  Instance_1           1.0  ...   600.402           ok
  24  Instance_1           1.0  ...  3600.000       memout
  25  Instance_1           1.0  ...  3600.000       memout
  26  Instance_1           1.0  ...    78.121           ok
  27  Instance_1           1.0  ...   600.402           ok
  28  Instance_1           1.0  ...  3600.000       memout
  29  Instance_1           1.0  ...  3600.000       memout
  ..         ...           ...  ...       ...          ...
  66  Instance_4           1.0  ...   422.530           ok
  67  Instance_4           1.0  ...  3600.000      timeout
  68  Instance_4           1.0  ...  3600.000       memout
  69  Instance_4           1.0  ...  3600.000       memout
  70  Instance_4           1.0  ...   422.530           ok
  71  Instance_4           1.0  ...  3600.000      timeout
  72  Instance_4           1.0  ...  3600.000       memout
  73  Instance_4           1.0  ...  3600.000       memout
  74  Instance_4           1.0  ...   422.530           ok
  75  Instance_4           1.0  ...  3600.000      timeout
  76  Instance_4           1.0  ...  3600.000       memout
  77  Instance_4           1.0  ...  3600.000       memout
  78  Instance_4           1.0  ...   422.530           ok
  79  Instance_4           1.0  ...  3600.000      timeout
  80  Instance_5           1.0  ...   182.019           ok
  81  Instance_5           1.0  ...  3600.000       memout
  82  Instance_5           1.0  ...    66.988           ok
  83  Instance_5           1.0  ...   249.488           ok
  84  Instance_5           1.0  ...   182.019           ok
  85  Instance_5           1.0  ...  3600.000       memout
  86  Instance_5           1.0  ...    66.988           ok
  87  Instance_5           1.0  ...   249.488           ok
  88  Instance_5           1.0  ...   182.019           ok
  89  Instance_5           1.0  ...  3600.000       memout
  90  Instance_5           1.0  ...    66.988           ok
  91  Instance_5           1.0  ...   249.488           ok
  92  Instance_5           1.0  ...   182.019           ok
  93  Instance_5           1.0  ...  3600.000       memout
  94  Instance_5           1.0  ...    66.988           ok
  95  Instance_5           1.0  ...   249.488           ok
  
  [96 rows x 9 columns],
  encoding: FacetedEncoding({
    color: Color({
      field: 'algorithm_y',
      legend: None,
      scale: Scale({
        domain: ['algorithm_0', 'algorithm_1', 'id_algorithm_0', 'id_algorithm_1'],
        range: ['#ffe200', '#f5ffff', '#6600db', '#11182b']
      }),
      type: 'nominal'
    })

In [0]:
alg_color_scheme = make_color_scheme(algSource, 'algorithm', four_colors)

alg_chart_colored = algChart.encode(color = alg_color_scheme)

alg_chart_colored

Chart({
  data:    instance_id  repetition       algorithm   runtime runstatus
  0   Instance_0         1.0     algorithm_0  3600.000    memout
  1   Instance_0         1.0     algorithm_1  3600.000    memout
  2   Instance_0         1.0  id_algorithm_0   311.863        ok
  3   Instance_0         1.0  id_algorithm_1  3600.000   timeout
  4   Instance_1         1.0     algorithm_0  3600.000    memout
  5   Instance_1         1.0     algorithm_1  3600.000    memout
  6   Instance_1         1.0  id_algorithm_0    78.121        ok
  7   Instance_1         1.0  id_algorithm_1   600.402        ok
  8   Instance_2         1.0     algorithm_0  3600.000    memout
  9   Instance_2         1.0     algorithm_1  3600.000    memout
  10  Instance_2         1.0  id_algorithm_0     0.012        ok
  11  Instance_2         1.0  id_algorithm_1    60.720        ok
  12  Instance_3         1.0     algorithm_0  3600.000    memout
  13  Instance_3         1.0     algorithm_1    92.986        ok
  14  Instance_3         1.0  id_algorithm_0  2031.039        ok
  15  Instance_3         1.0  id_algorithm_1   141.857        ok
  16  Instance_4         1.0     algorithm_0  3600.000    memout
  17  Instance_4         1.0     algorithm_1  3600.000    memout
  18  Instance_4         1.0  id_algorithm_0   422.530        ok
  19  Instance_4         1.0  id_algorithm_1  3600.000   timeout
  20  Instance_5         1.0     algorithm_0   182.019        ok
  21  Instance_5         1.0     algorithm_1  3600.000    memout
  22  Instance_5         1.0  id_algorithm_0    66.988        ok
  23  Instance_5         1.0  id_algorithm_1   249.488        ok,
  encoding: FacetedEncoding({
    color: Color({
      field: 'algorithm',
      legend: None,
      scale: Scale({
        domain: ['algorithm_0', 'algorithm_1', 'id_algorithm_0', 'id_algorithm_1'],
        range: ['#ffe200', '#f5ffff', '#6600db', '#11182b']
      }),
      type: 'nominal'
    }),
    x: X({
      field: 'algorithm',
      type: 'nominal'
    }),
    y: Y({
      aggregate: 'sum',
      field: 'runtime',
      type: 'quantitative'
    })
  }),
  mark: 'bar'
})

In [0]:
alg_selection = make_selection(['algorithm'])

with_select = algChart.add_selection(alg_selection).transform_filter(alg_selection)
with_select

Chart({
  data:    instance_id  repetition       algorithm   runtime runstatus
  0   Instance_0         1.0     algorithm_0  3600.000    memout
  1   Instance_0         1.0     algorithm_1  3600.000    memout
  2   Instance_0         1.0  id_algorithm_0   311.863        ok
  3   Instance_0         1.0  id_algorithm_1  3600.000   timeout
  4   Instance_1         1.0     algorithm_0  3600.000    memout
  5   Instance_1         1.0     algorithm_1  3600.000    memout
  6   Instance_1         1.0  id_algorithm_0    78.121        ok
  7   Instance_1         1.0  id_algorithm_1   600.402        ok
  8   Instance_2         1.0     algorithm_0  3600.000    memout
  9   Instance_2         1.0     algorithm_1  3600.000    memout
  10  Instance_2         1.0  id_algorithm_0     0.012        ok
  11  Instance_2         1.0  id_algorithm_1    60.720        ok
  12  Instance_3         1.0     algorithm_0  3600.000    memout
  13  Instance_3         1.0     algorithm_1    92.986        ok
  14  Instance_3         1.0  id_algorithm_0  2031.039        ok
  15  Instance_3         1.0  id_algorithm_1   141.857        ok
  16  Instance_4         1.0     algorithm_0  3600.000    memout
  17  Instance_4         1.0     algorithm_1  3600.000    memout
  18  Instance_4         1.0  id_algorithm_0   422.530        ok
  19  Instance_4         1.0  id_algorithm_1  3600.000   timeout
  20  Instance_5         1.0     algorithm_0   182.019        ok
  21  Instance_5         1.0     algorithm_1  3600.000    memout
  22  Instance_5         1.0  id_algorithm_0    66.988        ok
  23  Instance_5         1.0  id_algorithm_1   249.488        ok,
  encoding: FacetedEncoding({
    x: X({
      field: 'algorithm',
      type: 'nominal'
    }),
    y: Y({
      aggregate: 'sum',
      field: 'runtime',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  selection: {'selector009': SelectionDef({
    fields: ['algorithm'],
    type: 'single'
  })},
  transform: [FilterTransform({
    filter: {'selection': 'selector009'}
  })]
})

In [0]:
alg_selector = make_selection(['algorithm'])
alg_color = make_color_scheme(algSource, 'algorithm', four_colors)
alg_color_update = make_color_condition(alg_selector, alg_color)

fresh_chart = algChart.add_selection(alg_selector).encode(color = alg_color_update)

fresh_chart

Chart({
  data:    instance_id  repetition       algorithm   runtime runstatus
  0   Instance_0         1.0     algorithm_0  3600.000    memout
  1   Instance_0         1.0     algorithm_1  3600.000    memout
  2   Instance_0         1.0  id_algorithm_0   311.863        ok
  3   Instance_0         1.0  id_algorithm_1  3600.000   timeout
  4   Instance_1         1.0     algorithm_0  3600.000    memout
  5   Instance_1         1.0     algorithm_1  3600.000    memout
  6   Instance_1         1.0  id_algorithm_0    78.121        ok
  7   Instance_1         1.0  id_algorithm_1   600.402        ok
  8   Instance_2         1.0     algorithm_0  3600.000    memout
  9   Instance_2         1.0     algorithm_1  3600.000    memout
  10  Instance_2         1.0  id_algorithm_0     0.012        ok
  11  Instance_2         1.0  id_algorithm_1    60.720        ok
  12  Instance_3         1.0     algorithm_0  3600.000    memout
  13  Instance_3         1.0     algorithm_1    92.986        ok
  14  Instance_3         1.0  id_algorithm_0  2031.039        ok
  15  Instance_3         1.0  id_algorithm_1   141.857        ok
  16  Instance_4         1.0     algorithm_0  3600.000    memout
  17  Instance_4         1.0     algorithm_1  3600.000    memout
  18  Instance_4         1.0  id_algorithm_0   422.530        ok
  19  Instance_4         1.0  id_algorithm_1  3600.000   timeout
  20  Instance_5         1.0     algorithm_0   182.019        ok
  21  Instance_5         1.0     algorithm_1  3600.000    memout
  22  Instance_5         1.0  id_algorithm_0    66.988        ok
  23  Instance_5         1.0  id_algorithm_1   249.488        ok,
  encoding: FacetedEncoding({
    color: ColorValue({
      condition: Color({
        field: FieldName('algorithm'),
        legend: None,
        scale: Scale({
          domain: ['algorithm_0', 'algorithm_1', 'id_algorithm_0', 'id_algorithm_1'],
          range: ['#ffe200', '#f5ffff', '#6600db', '#11182b']
        }),
        selection: 'selector047',
        type: StandardType('nominal')
      }),
      value: 'darkgray'
    }),
    x: X({
      field: 'algorithm',
      type: 'nominal'
    }),
    y: Y({
      aggregate: 'sum',
      field: 'runtime',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  selection: {'selector047': SelectionDef({
    fields: ['algorithm'],
    type: 'single'
  })}
})

In [0]:
colorable_chart = apply_selection_color(algChart, alg_selector, alg_color_update)
                                        
colorable_chart

Chart({
  data:    instance_id  repetition       algorithm   runtime runstatus
  0   Instance_0         1.0     algorithm_0  3600.000    memout
  1   Instance_0         1.0     algorithm_1  3600.000    memout
  2   Instance_0         1.0  id_algorithm_0   311.863        ok
  3   Instance_0         1.0  id_algorithm_1  3600.000   timeout
  4   Instance_1         1.0     algorithm_0  3600.000    memout
  5   Instance_1         1.0     algorithm_1  3600.000    memout
  6   Instance_1         1.0  id_algorithm_0    78.121        ok
  7   Instance_1         1.0  id_algorithm_1   600.402        ok
  8   Instance_2         1.0     algorithm_0  3600.000    memout
  9   Instance_2         1.0     algorithm_1  3600.000    memout
  10  Instance_2         1.0  id_algorithm_0     0.012        ok
  11  Instance_2         1.0  id_algorithm_1    60.720        ok
  12  Instance_3         1.0     algorithm_0  3600.000    memout
  13  Instance_3         1.0     algorithm_1    92.986        ok
  14  Instance_3         1.0  id_algorithm_0  2031.039        ok
  15  Instance_3         1.0  id_algorithm_1   141.857        ok
  16  Instance_4         1.0     algorithm_0  3600.000    memout
  17  Instance_4         1.0     algorithm_1  3600.000    memout
  18  Instance_4         1.0  id_algorithm_0   422.530        ok
  19  Instance_4         1.0  id_algorithm_1  3600.000   timeout
  20  Instance_5         1.0     algorithm_0   182.019        ok
  21  Instance_5         1.0     algorithm_1  3600.000    memout
  22  Instance_5         1.0  id_algorithm_0    66.988        ok
  23  Instance_5         1.0  id_algorithm_1   249.488        ok,
  encoding: FacetedEncoding({
    color: ColorValue({
      condition: Color({
        field: FieldName('algorithm'),
        legend: None,
        scale: Scale({
          domain: ['algorithm_0', 'algorithm_1', 'id_algorithm_0', 'id_algorithm_1'],
          range: ['#ffe200', '#f5ffff', '#6600db', '#11182b']
        }),
        selection: 'selector047',
        type: StandardType('nominal')
      }),
      value: 'darkgray'
    }),
    x: X({
      field: 'algorithm',
      type: 'nominal'
    }),
    y: Y({
      aggregate: 'sum',
      field: 'runtime',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  selection: {'selector047': SelectionDef({
    fields: ['algorithm'],
    type: 'single'
  })}
})

In [0]:
legend_base = make_graph(alg_joined, 'runstatus_x', 'runstatus_y', 'rect')
legend_select = make_selection(['runstatus_x', 'runstatus_y'], True)
legend_colors = make_color_scheme(alg_joined, 'runstatus_y', three_colors)
legend_color_update = make_color_condition(legend_select, legend_colors, alt.value('darkgray'))
legend = apply_selection_color(legend_base, legend_select, legend_color_update)

legend

Chart({
  data:    instance_id  repetition_x  ... runtime_y  runstatus_y
  0   Instance_0           1.0  ...  3600.000       memout
  1   Instance_0           1.0  ...  3600.000       memout
  2   Instance_0           1.0  ...   311.863           ok
  3   Instance_0           1.0  ...  3600.000      timeout
  4   Instance_0           1.0  ...  3600.000       memout
  5   Instance_0           1.0  ...  3600.000       memout
  6   Instance_0           1.0  ...   311.863           ok
  7   Instance_0           1.0  ...  3600.000      timeout
  8   Instance_0           1.0  ...  3600.000       memout
  9   Instance_0           1.0  ...  3600.000       memout
  10  Instance_0           1.0  ...   311.863           ok
  11  Instance_0           1.0  ...  3600.000      timeout
  12  Instance_0           1.0  ...  3600.000       memout
  13  Instance_0           1.0  ...  3600.000       memout
  14  Instance_0           1.0  ...   311.863           ok
  15  Instance_0           1.0  ...  3600.000      timeout
  16  Instance_1           1.0  ...  3600.000       memout
  17  Instance_1           1.0  ...  3600.000       memout
  18  Instance_1           1.0  ...    78.121           ok
  19  Instance_1           1.0  ...   600.402           ok
  20  Instance_1           1.0  ...  3600.000       memout
  21  Instance_1           1.0  ...  3600.000       memout
  22  Instance_1           1.0  ...    78.121           ok
  23  Instance_1           1.0  ...   600.402           ok
  24  Instance_1           1.0  ...  3600.000       memout
  25  Instance_1           1.0  ...  3600.000       memout
  26  Instance_1           1.0  ...    78.121           ok
  27  Instance_1           1.0  ...   600.402           ok
  28  Instance_1           1.0  ...  3600.000       memout
  29  Instance_1           1.0  ...  3600.000       memout
  ..         ...           ...  ...       ...          ...
  66  Instance_4           1.0  ...   422.530           ok
  67  Instance_4           1.0  ...  3600.000      timeout
  68  Instance_4           1.0  ...  3600.000       memout
  69  Instance_4           1.0  ...  3600.000       memout
  70  Instance_4           1.0  ...   422.530           ok
  71  Instance_4           1.0  ...  3600.000      timeout
  72  Instance_4           1.0  ...  3600.000       memout
  73  Instance_4           1.0  ...  3600.000       memout
  74  Instance_4           1.0  ...   422.530           ok
  75  Instance_4           1.0  ...  3600.000      timeout
  76  Instance_4           1.0  ...  3600.000       memout
  77  Instance_4           1.0  ...  3600.000       memout
  78  Instance_4           1.0  ...   422.530           ok
  79  Instance_4           1.0  ...  3600.000      timeout
  80  Instance_5           1.0  ...   182.019           ok
  81  Instance_5           1.0  ...  3600.000       memout
  82  Instance_5           1.0  ...    66.988           ok
  83  Instance_5           1.0  ...   249.488           ok
  84  Instance_5           1.0  ...   182.019           ok
  85  Instance_5           1.0  ...  3600.000       memout
  86  Instance_5           1.0  ...    66.988           ok
  87  Instance_5           1.0  ...   249.488           ok
  88  Instance_5           1.0  ...   182.019           ok
  89  Instance_5           1.0  ...  3600.000       memout
  90  Instance_5           1.0  ...    66.988           ok
  91  Instance_5           1.0  ...   249.488           ok
  92  Instance_5           1.0  ...   182.019           ok
  93  Instance_5           1.0  ...  3600.000       memout
  94  Instance_5           1.0  ...    66.988           ok
  95  Instance_5           1.0  ...   249.488           ok
  
  [96 rows x 9 columns],
  encoding: FacetedEncoding({
    color: ColorValue({
      condition: Color({
        field: FieldName('runstatus_y'),
        legend: None,
        scale: Scale({
          domain: ['memout', 'ok', 'timeout'],
          range: ['#dd1f75', '#fda900', '#2a4dac']
        }),
        selection: 'selec

In [0]:
a_leg_base = make_graph(alg_joined, 'algorithm_x', 'algorithm_y', 'rect')
a_leg_select = make_selection(['algorithm_x', 'algorithm_y'])
a_leg_colors = make_color_scheme(alg_joined, 'algorithm_y', four_colors)
a_leg_col_update = make_color_condition(a_leg_select, a_leg_colors)
alg_legend = apply_selection_color(a_leg_base, a_leg_select, a_leg_col_update)

alg_legend

Chart({
  data:    instance_id  repetition_x  ... runtime_y  runstatus_y
  0   Instance_0           1.0  ...  3600.000       memout
  1   Instance_0           1.0  ...  3600.000       memout
  2   Instance_0           1.0  ...   311.863           ok
  3   Instance_0           1.0  ...  3600.000      timeout
  4   Instance_0           1.0  ...  3600.000       memout
  5   Instance_0           1.0  ...  3600.000       memout
  6   Instance_0           1.0  ...   311.863           ok
  7   Instance_0           1.0  ...  3600.000      timeout
  8   Instance_0           1.0  ...  3600.000       memout
  9   Instance_0           1.0  ...  3600.000       memout
  10  Instance_0           1.0  ...   311.863           ok
  11  Instance_0           1.0  ...  3600.000      timeout
  12  Instance_0           1.0  ...  3600.000       memout
  13  Instance_0           1.0  ...  3600.000       memout
  14  Instance_0           1.0  ...   311.863           ok
  15  Instance_0           1.0  ...  3600.000      timeout
  16  Instance_1           1.0  ...  3600.000       memout
  17  Instance_1           1.0  ...  3600.000       memout
  18  Instance_1           1.0  ...    78.121           ok
  19  Instance_1           1.0  ...   600.402           ok
  20  Instance_1           1.0  ...  3600.000       memout
  21  Instance_1           1.0  ...  3600.000       memout
  22  Instance_1           1.0  ...    78.121           ok
  23  Instance_1           1.0  ...   600.402           ok
  24  Instance_1           1.0  ...  3600.000       memout
  25  Instance_1           1.0  ...  3600.000       memout
  26  Instance_1           1.0  ...    78.121           ok
  27  Instance_1           1.0  ...   600.402           ok
  28  Instance_1           1.0  ...  3600.000       memout
  29  Instance_1           1.0  ...  3600.000       memout
  ..         ...           ...  ...       ...          ...
  66  Instance_4           1.0  ...   422.530           ok
  67  Instance_4           1.0  ...  3600.000      timeout
  68  Instance_4           1.0  ...  3600.000       memout
  69  Instance_4           1.0  ...  3600.000       memout
  70  Instance_4           1.0  ...   422.530           ok
  71  Instance_4           1.0  ...  3600.000      timeout
  72  Instance_4           1.0  ...  3600.000       memout
  73  Instance_4           1.0  ...  3600.000       memout
  74  Instance_4           1.0  ...   422.530           ok
  75  Instance_4           1.0  ...  3600.000      timeout
  76  Instance_4           1.0  ...  3600.000       memout
  77  Instance_4           1.0  ...  3600.000       memout
  78  Instance_4           1.0  ...   422.530           ok
  79  Instance_4           1.0  ...  3600.000      timeout
  80  Instance_5           1.0  ...   182.019           ok
  81  Instance_5           1.0  ...  3600.000       memout
  82  Instance_5           1.0  ...    66.988           ok
  83  Instance_5           1.0  ...   249.488           ok
  84  Instance_5           1.0  ...   182.019           ok
  85  Instance_5           1.0  ...  3600.000       memout
  86  Instance_5           1.0  ...    66.988           ok
  87  Instance_5           1.0  ...   249.488           ok
  88  Instance_5           1.0  ...   182.019           ok
  89  Instance_5           1.0  ...  3600.000       memout
  90  Instance_5           1.0  ...    66.988           ok
  91  Instance_5           1.0  ...   249.488           ok
  92  Instance_5           1.0  ...   182.019           ok
  93  Instance_5           1.0  ...  3600.000       memout
  94  Instance_5           1.0  ...    66.988           ok
  95  Instance_5           1.0  ...   249.488           ok
  
  [96 rows x 9 columns],
  encoding: FacetedEncoding({
    color: ColorValue({
      condition: Color({
        field: FieldName('algorithm_y'),
        legend: None,
        scale: Scale({
          domain: ['algorithm_0', 'algorithm_1', 'id_algorithm_0', 'id_algorithm_1'],
          range: ['#ffe200', '#f5ffff', '#6600db'

In [0]:
scatter_base = make_graph(alg_joined, 'runtime_x', 'runtime_y')
scatter_filtered = scatter_base.transform_filter(
                      legend_select
                   ).transform_filter(
                      a_leg_select
                   ).interactive()

(alg_legend & legend) | scatter_filtered

HConcatChart({
  data:    instance_id  repetition_x  ... runtime_y  runstatus_y
  0   Instance_0           1.0  ...  3600.000       memout
  1   Instance_0           1.0  ...  3600.000       memout
  2   Instance_0           1.0  ...   311.863           ok
  3   Instance_0           1.0  ...  3600.000      timeout
  4   Instance_0           1.0  ...  3600.000       memout
  5   Instance_0           1.0  ...  3600.000       memout
  6   Instance_0           1.0  ...   311.863           ok
  7   Instance_0           1.0  ...  3600.000      timeout
  8   Instance_0           1.0  ...  3600.000       memout
  9   Instance_0           1.0  ...  3600.000       memout
  10  Instance_0           1.0  ...   311.863           ok
  11  Instance_0           1.0  ...  3600.000      timeout
  12  Instance_0           1.0  ...  3600.000       memout
  13  Instance_0           1.0  ...  3600.000       memout
  14  Instance_0           1.0  ...   311.863           ok
  15  Instance_0           1.0  ...  3600.000      timeout
  16  Instance_1           1.0  ...  3600.000       memout
  17  Instance_1           1.0  ...  3600.000       memout
  18  Instance_1           1.0  ...    78.121           ok
  19  Instance_1           1.0  ...   600.402           ok
  20  Instance_1           1.0  ...  3600.000       memout
  21  Instance_1           1.0  ...  3600.000       memout
  22  Instance_1           1.0  ...    78.121           ok
  23  Instance_1           1.0  ...   600.402           ok
  24  Instance_1           1.0  ...  3600.000       memout
  25  Instance_1           1.0  ...  3600.000       memout
  26  Instance_1           1.0  ...    78.121           ok
  27  Instance_1           1.0  ...   600.402           ok
  28  Instance_1           1.0  ...  3600.000       memout
  29  Instance_1           1.0  ...  3600.000       memout
  ..         ...           ...  ...       ...          ...
  66  Instance_4           1.0  ...   422.530           ok
  67  Instance_4           1.0  ...  3600.000      timeout
  68  Instance_4           1.0  ...  3600.000       memout
  69  Instance_4           1.0  ...  3600.000       memout
  70  Instance_4           1.0  ...   422.530           ok
  71  Instance_4           1.0  ...  3600.000      timeout
  72  Instance_4           1.0  ...  3600.000       memout
  73  Instance_4           1.0  ...  3600.000       memout
  74  Instance_4           1.0  ...   422.530           ok
  75  Instance_4           1.0  ...  3600.000      timeout
  76  Instance_4           1.0  ...  3600.000       memout
  77  Instance_4           1.0  ...  3600.000       memout
  78  Instance_4           1.0  ...   422.530           ok
  79  Instance_4           1.0  ...  3600.000      timeout
  80  Instance_5           1.0  ...   182.019           ok
  81  Instance_5           1.0  ...  3600.000       memout
  82  Instance_5           1.0  ...    66.988           ok
  83  Instance_5           1.0  ...   249.488           ok
  84  Instance_5           1.0  ...   182.019           ok
  85  Instance_5           1.0  ...  3600.000       memout
  86  Instance_5           1.0  ...    66.988           ok
  87  Instance_5           1.0  ...   249.488           ok
  88  Instance_5           1.0  ...   182.019           ok
  89  Instance_5           1.0  ...  3600.000       memout
  90  Instance_5           1.0  ...    66.988           ok
  91  Instance_5           1.0  ...   249.488           ok
  92  Instance_5           1.0  ...   182.019           ok
  93  Instance_5           1.0  ...  3600.000       memout
  94  Instance_5           1.0  ...    66.988           ok
  95  Instance_5           1.0  ...   249.488           ok
  
  [96 rows x 9 columns],
  hconcat: [VConcatChart({
    vconcat: [Chart({
      encoding: FacetedEncoding({
        color: ColorValue({
          condition: Color({
            field: FieldName('algorithm_y'),
            legend: None,
            scale: Scale({
              domain: ['algorithm_0', 'algorithm_1', 'i

In [0]:
create_comparative_scatter(alg_joined, 'instance_id', 'algorithm_x', 'algorithm_y',
                           'runtime_x', 'runtime_y', 'runstatus_x', 
                           'runstatus_y', four_colors)

HConcatChart({
  data:    instance_id  repetition_x  ... runtime_y  runstatus_y
  0   Instance_0           1.0  ...  3600.000       memout
  1   Instance_0           1.0  ...  3600.000       memout
  2   Instance_0           1.0  ...   311.863           ok
  3   Instance_0           1.0  ...  3600.000      timeout
  4   Instance_0           1.0  ...  3600.000       memout
  5   Instance_0           1.0  ...  3600.000       memout
  6   Instance_0           1.0  ...   311.863           ok
  7   Instance_0           1.0  ...  3600.000      timeout
  8   Instance_0           1.0  ...  3600.000       memout
  9   Instance_0           1.0  ...  3600.000       memout
  10  Instance_0           1.0  ...   311.863           ok
  11  Instance_0           1.0  ...  3600.000      timeout
  12  Instance_0           1.0  ...  3600.000       memout
  13  Instance_0           1.0  ...  3600.000       memout
  14  Instance_0           1.0  ...   311.863           ok
  15  Instance_0           1.0  ...  3600.000      timeout
  16  Instance_1           1.0  ...  3600.000       memout
  17  Instance_1           1.0  ...  3600.000       memout
  18  Instance_1           1.0  ...    78.121           ok
  19  Instance_1           1.0  ...   600.402           ok
  20  Instance_1           1.0  ...  3600.000       memout
  21  Instance_1           1.0  ...  3600.000       memout
  22  Instance_1           1.0  ...    78.121           ok
  23  Instance_1           1.0  ...   600.402           ok
  24  Instance_1           1.0  ...  3600.000       memout
  25  Instance_1           1.0  ...  3600.000       memout
  26  Instance_1           1.0  ...    78.121           ok
  27  Instance_1           1.0  ...   600.402           ok
  28  Instance_1           1.0  ...  3600.000       memout
  29  Instance_1           1.0  ...  3600.000       memout
  ..         ...           ...  ...       ...          ...
  66  Instance_4           1.0  ...   422.530           ok
  67  Instance_4           1.0  ...  3600.000      timeout
  68  Instance_4           1.0  ...  3600.000       memout
  69  Instance_4           1.0  ...  3600.000       memout
  70  Instance_4           1.0  ...   422.530           ok
  71  Instance_4           1.0  ...  3600.000      timeout
  72  Instance_4           1.0  ...  3600.000       memout
  73  Instance_4           1.0  ...  3600.000       memout
  74  Instance_4           1.0  ...   422.530           ok
  75  Instance_4           1.0  ...  3600.000      timeout
  76  Instance_4           1.0  ...  3600.000       memout
  77  Instance_4           1.0  ...  3600.000       memout
  78  Instance_4           1.0  ...   422.530           ok
  79  Instance_4           1.0  ...  3600.000      timeout
  80  Instance_5           1.0  ...   182.019           ok
  81  Instance_5           1.0  ...  3600.000       memout
  82  Instance_5           1.0  ...    66.988           ok
  83  Instance_5           1.0  ...   249.488           ok
  84  Instance_5           1.0  ...   182.019           ok
  85  Instance_5           1.0  ...  3600.000       memout
  86  Instance_5           1.0  ...    66.988           ok
  87  Instance_5           1.0  ...   249.488           ok
  88  Instance_5           1.0  ...   182.019           ok
  89  Instance_5           1.0  ...  3600.000       memout
  90  Instance_5           1.0  ...    66.988           ok
  91  Instance_5           1.0  ...   249.488           ok
  92  Instance_5           1.0  ...   182.019           ok
  93  Instance_5           1.0  ...  3600.000       memout
  94  Instance_5           1.0  ...    66.988           ok
  95  Instance_5           1.0  ...   249.488           ok
  
  [96 rows x 9 columns],
  hconcat: [VConcatChart({
    vconcat: [Chart({
      encoding: FacetedEncoding({
        color: ColorValue({
          condition: Color({
            field: FieldName('algorithm_y'),
            legend: None,
            scale: Scale({
              domain: ['algorithm_0', 'algorithm_1', 'i

In [0]:
def save_scatter(chart, name='chart'):
  chart.save(name+'.html')

In [0]:
sample_scatter = create_comparative_scatter(alg_joined, 'algorithm_x', 'algorithm_y', 'runtime_x', 'runtime_y', 
                                            'runstatus_x', 'runstatus_y', four_colors)

In [0]:
sample_scatter

HConcatChart({
  data:    instance_id  repetition_x  ... runtime_y  runstatus_y
  0   Instance_0           1.0  ...  3600.000       memout
  1   Instance_0           1.0  ...  3600.000       memout
  2   Instance_0           1.0  ...   311.863           ok
  3   Instance_0           1.0  ...  3600.000      timeout
  4   Instance_0           1.0  ...  3600.000       memout
  5   Instance_0           1.0  ...  3600.000       memout
  6   Instance_0           1.0  ...   311.863           ok
  7   Instance_0           1.0  ...  3600.000      timeout
  8   Instance_0           1.0  ...  3600.000       memout
  9   Instance_0           1.0  ...  3600.000       memout
  10  Instance_0           1.0  ...   311.863           ok
  11  Instance_0           1.0  ...  3600.000      timeout
  12  Instance_0           1.0  ...  3600.000       memout
  13  Instance_0           1.0  ...  3600.000       memout
  14  Instance_0           1.0  ...   311.863           ok
  15  Instance_0           1.0  ...  3600.000      timeout
  16  Instance_1           1.0  ...  3600.000       memout
  17  Instance_1           1.0  ...  3600.000       memout
  18  Instance_1           1.0  ...    78.121           ok
  19  Instance_1           1.0  ...   600.402           ok
  20  Instance_1           1.0  ...  3600.000       memout
  21  Instance_1           1.0  ...  3600.000       memout
  22  Instance_1           1.0  ...    78.121           ok
  23  Instance_1           1.0  ...   600.402           ok
  24  Instance_1           1.0  ...  3600.000       memout
  25  Instance_1           1.0  ...  3600.000       memout
  26  Instance_1           1.0  ...    78.121           ok
  27  Instance_1           1.0  ...   600.402           ok
  28  Instance_1           1.0  ...  3600.000       memout
  29  Instance_1           1.0  ...  3600.000       memout
  ..         ...           ...  ...       ...          ...
  66  Instance_4           1.0  ...   422.530           ok
  67  Instance_4           1.0  ...  3600.000      timeout
  68  Instance_4           1.0  ...  3600.000       memout
  69  Instance_4           1.0  ...  3600.000       memout
  70  Instance_4           1.0  ...   422.530           ok
  71  Instance_4           1.0  ...  3600.000      timeout
  72  Instance_4           1.0  ...  3600.000       memout
  73  Instance_4           1.0  ...  3600.000       memout
  74  Instance_4           1.0  ...   422.530           ok
  75  Instance_4           1.0  ...  3600.000      timeout
  76  Instance_4           1.0  ...  3600.000       memout
  77  Instance_4           1.0  ...  3600.000       memout
  78  Instance_4           1.0  ...   422.530           ok
  79  Instance_4           1.0  ...  3600.000      timeout
  80  Instance_5           1.0  ...   182.019           ok
  81  Instance_5           1.0  ...  3600.000       memout
  82  Instance_5           1.0  ...    66.988           ok
  83  Instance_5           1.0  ...   249.488           ok
  84  Instance_5           1.0  ...   182.019           ok
  85  Instance_5           1.0  ...  3600.000       memout
  86  Instance_5           1.0  ...    66.988           ok
  87  Instance_5           1.0  ...   249.488           ok
  88  Instance_5           1.0  ...   182.019           ok
  89  Instance_5           1.0  ...  3600.000       memout
  90  Instance_5           1.0  ...    66.988           ok
  91  Instance_5           1.0  ...   249.488           ok
  92  Instance_5           1.0  ...   182.019           ok
  93  Instance_5           1.0  ...  3600.000       memout
  94  Instance_5           1.0  ...    66.988           ok
  95  Instance_5           1.0  ...   249.488           ok
  
  [96 rows x 9 columns],
  hconcat: [VConcatChart({
    vconcat: [Chart({
      encoding: FacetedEncoding({
        color: ColorValue({
          condition: Color({
            field: FieldName('algorithm_y'),
            legend: None,
            scale: Scale({
              domain: ['algorithm_0', 'algorithm_1', 'i

In [0]:
save_scatter(sample_scatter, 'sample scatter')

In [0]:
tester = make_graph(alg_joined, 'runtime_x', 'runtime_y')
tester

Chart({
  data:    instance_id  repetition_x  ... runtime_y  runstatus_y
  0   Instance_0           1.0  ...  3600.000       memout
  1   Instance_0           1.0  ...  3600.000       memout
  2   Instance_0           1.0  ...   311.863           ok
  3   Instance_0           1.0  ...  3600.000      timeout
  4   Instance_0           1.0  ...  3600.000       memout
  5   Instance_0           1.0  ...  3600.000       memout
  6   Instance_0           1.0  ...   311.863           ok
  7   Instance_0           1.0  ...  3600.000      timeout
  8   Instance_0           1.0  ...  3600.000       memout
  9   Instance_0           1.0  ...  3600.000       memout
  10  Instance_0           1.0  ...   311.863           ok
  11  Instance_0           1.0  ...  3600.000      timeout
  12  Instance_0           1.0  ...  3600.000       memout
  13  Instance_0           1.0  ...  3600.000       memout
  14  Instance_0           1.0  ...   311.863           ok
  15  Instance_0           1.0  ...  3600.000      timeout
  16  Instance_1           1.0  ...  3600.000       memout
  17  Instance_1           1.0  ...  3600.000       memout
  18  Instance_1           1.0  ...    78.121           ok
  19  Instance_1           1.0  ...   600.402           ok
  20  Instance_1           1.0  ...  3600.000       memout
  21  Instance_1           1.0  ...  3600.000       memout
  22  Instance_1           1.0  ...    78.121           ok
  23  Instance_1           1.0  ...   600.402           ok
  24  Instance_1           1.0  ...  3600.000       memout
  25  Instance_1           1.0  ...  3600.000       memout
  26  Instance_1           1.0  ...    78.121           ok
  27  Instance_1           1.0  ...   600.402           ok
  28  Instance_1           1.0  ...  3600.000       memout
  29  Instance_1           1.0  ...  3600.000       memout
  ..         ...           ...  ...       ...          ...
  66  Instance_4           1.0  ...   422.530           ok
  67  Instance_4           1.0  ...  3600.000      timeout
  68  Instance_4           1.0  ...  3600.000       memout
  69  Instance_4           1.0  ...  3600.000       memout
  70  Instance_4           1.0  ...   422.530           ok
  71  Instance_4           1.0  ...  3600.000      timeout
  72  Instance_4           1.0  ...  3600.000       memout
  73  Instance_4           1.0  ...  3600.000       memout
  74  Instance_4           1.0  ...   422.530           ok
  75  Instance_4           1.0  ...  3600.000      timeout
  76  Instance_4           1.0  ...  3600.000       memout
  77  Instance_4           1.0  ...  3600.000       memout
  78  Instance_4           1.0  ...   422.530           ok
  79  Instance_4           1.0  ...  3600.000      timeout
  80  Instance_5           1.0  ...   182.019           ok
  81  Instance_5           1.0  ...  3600.000       memout
  82  Instance_5           1.0  ...    66.988           ok
  83  Instance_5           1.0  ...   249.488           ok
  84  Instance_5           1.0  ...   182.019           ok
  85  Instance_5           1.0  ...  3600.000       memout
  86  Instance_5           1.0  ...    66.988           ok
  87  Instance_5           1.0  ...   249.488           ok
  88  Instance_5           1.0  ...   182.019           ok
  89  Instance_5           1.0  ...  3600.000       memout
  90  Instance_5           1.0  ...    66.988           ok
  91  Instance_5           1.0  ...   249.488           ok
  92  Instance_5           1.0  ...   182.019           ok
  93  Instance_5           1.0  ...  3600.000       memout
  94  Instance_5           1.0  ...    66.988           ok
  95  Instance_5           1.0  ...   249.488           ok
  
  [96 rows x 9 columns],
  encoding: FacetedEncoding({
    x: X({
      field: 'runtime_x',
      type: 'quantitative'
    }),
    y: Y({
      field: 'runtime_y',
      type: 'quantitative'
    })
  }),
  mark: MarkDef({
    size: 60,
    type: 'circle'
  })
})

In [0]:
test_tip = tester.encode(tooltip = ['instance_id'])
test_tip

Chart({
  data:    instance_id  repetition_x  ... runtime_y  runstatus_y
  0   Instance_0           1.0  ...  3600.000       memout
  1   Instance_0           1.0  ...  3600.000       memout
  2   Instance_0           1.0  ...   311.863           ok
  3   Instance_0           1.0  ...  3600.000      timeout
  4   Instance_0           1.0  ...  3600.000       memout
  5   Instance_0           1.0  ...  3600.000       memout
  6   Instance_0           1.0  ...   311.863           ok
  7   Instance_0           1.0  ...  3600.000      timeout
  8   Instance_0           1.0  ...  3600.000       memout
  9   Instance_0           1.0  ...  3600.000       memout
  10  Instance_0           1.0  ...   311.863           ok
  11  Instance_0           1.0  ...  3600.000      timeout
  12  Instance_0           1.0  ...  3600.000       memout
  13  Instance_0           1.0  ...  3600.000       memout
  14  Instance_0           1.0  ...   311.863           ok
  15  Instance_0           1.0  ...  3600.000      timeout
  16  Instance_1           1.0  ...  3600.000       memout
  17  Instance_1           1.0  ...  3600.000       memout
  18  Instance_1           1.0  ...    78.121           ok
  19  Instance_1           1.0  ...   600.402           ok
  20  Instance_1           1.0  ...  3600.000       memout
  21  Instance_1           1.0  ...  3600.000       memout
  22  Instance_1           1.0  ...    78.121           ok
  23  Instance_1           1.0  ...   600.402           ok
  24  Instance_1           1.0  ...  3600.000       memout
  25  Instance_1           1.0  ...  3600.000       memout
  26  Instance_1           1.0  ...    78.121           ok
  27  Instance_1           1.0  ...   600.402           ok
  28  Instance_1           1.0  ...  3600.000       memout
  29  Instance_1           1.0  ...  3600.000       memout
  ..         ...           ...  ...       ...          ...
  66  Instance_4           1.0  ...   422.530           ok
  67  Instance_4           1.0  ...  3600.000      timeout
  68  Instance_4           1.0  ...  3600.000       memout
  69  Instance_4           1.0  ...  3600.000       memout
  70  Instance_4           1.0  ...   422.530           ok
  71  Instance_4           1.0  ...  3600.000      timeout
  72  Instance_4           1.0  ...  3600.000       memout
  73  Instance_4           1.0  ...  3600.000       memout
  74  Instance_4           1.0  ...   422.530           ok
  75  Instance_4           1.0  ...  3600.000      timeout
  76  Instance_4           1.0  ...  3600.000       memout
  77  Instance_4           1.0  ...  3600.000       memout
  78  Instance_4           1.0  ...   422.530           ok
  79  Instance_4           1.0  ...  3600.000      timeout
  80  Instance_5           1.0  ...   182.019           ok
  81  Instance_5           1.0  ...  3600.000       memout
  82  Instance_5           1.0  ...    66.988           ok
  83  Instance_5           1.0  ...   249.488           ok
  84  Instance_5           1.0  ...   182.019           ok
  85  Instance_5           1.0  ...  3600.000       memout
  86  Instance_5           1.0  ...    66.988           ok
  87  Instance_5           1.0  ...   249.488           ok
  88  Instance_5           1.0  ...   182.019           ok
  89  Instance_5           1.0  ...  3600.000       memout
  90  Instance_5           1.0  ...    66.988           ok
  91  Instance_5           1.0  ...   249.488           ok
  92  Instance_5           1.0  ...   182.019           ok
  93  Instance_5           1.0  ...  3600.000       memout
  94  Instance_5           1.0  ...    66.988           ok
  95  Instance_5           1.0  ...   249.488           ok
  
  [96 rows x 9 columns],
  encoding: FacetedEncoding({
    tooltip: [Tooltip({
      field: 'instance_id',
      type: 'nominal'
    })],
    x: X({
      field: 'runtime_x',
      type: 'quantitative'
    }),
    y: Y({
      field: 'runtime_y',
      type: 'quantitative'
    })
  }),
  mark: MarkDef({
    size: 60,
    ty

In [0]:
url = 'alg_joined.json'
alg_joined.to_json(url, orient='records')

json_scatter = create_comparative_scatter(url, 'instance_id', 'algorithm_x', 'algorithm_y', 'runtime_x', 'runtime_y', 
                                            'runstatus_x', 'runstatus_y', four_colors)

TypeError: ignored

In [0]:
corgi_in = pd.read_csv('corgi_named.csv', index_col = None)

corgi_chart = alt.Chart(corgi_in).mark_bar().encode(
    x = 'Name',
    y = 'Cuteness',
    color = alt.Color('Name', legend=None)
)

corgi_chart

Chart({
  data:                       Name  ... Cuteness
  0                    Holly  ...        7
  1                    Ralph  ...        7
  2  Molly ("Thing of Evil")  ...        9
  3                      Ein  ...       11
  4                     Loki  ...        8
  5                    Rufus  ...        9
  6             Sutter Brown  ...        8
  7               Hope Corgi  ...       11
  
  [8 rows x 4 columns],
  encoding: FacetedEncoding({
    color: Color({
      field: 'Name',
      legend: None,
      type: 'nominal'
    }),
    x: X({
      field: 'Name',
      type: 'nominal'
    }),
    y: Y({
      field: 'Cuteness',
      type: 'quantitative'
    })
  }),
  mark: 'bar'
})

In [0]:
url = 'corgi_in.json'
corgi_in.to_json(url, orient = 'records')

corgi_chart_from_url = alt.Chart(url).mark_line().encode(
    x = 'Name:N',
    y = 'Cuteness:Q'
)

corgi_chart_from_url

Chart({
  data: 'corgi_in.json',
  encoding: FacetedEncoding({
    x: X({
      field: 'Name',
      type: 'nominal'
    }),
    y: Y({
      field: 'Cuteness',
      type: 'quantitative'
    })
  }),
  mark: 'line'
})

In [0]:
import altair as alt
from vega_datasets import data
url = data.cars.url

alt.Chart(url).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q'
)

Chart({
  data: 'https://vega.github.io/vega-datasets/data/cars.json',
  encoding: FacetedEncoding({
    x: X({
      field: 'Horsepower',
      type: 'quantitative'
    }),
    y: Y({
      field: 'Miles_per_Gallon',
      type: 'quantitative'
    })
  }),
  mark: 'point'
})

In [0]:
file_name = 'corgi_in.json'

alt.Chart(file_name).mark_point().encode(
    x='Name:N',
    y = 'Cuteness:Q'
)

Chart({
  data: 'corgi_in.json',
  encoding: FacetedEncoding({
    x: X({
      field: 'Name',
      type: 'nominal'
    }),
    y: Y({
      field: 'Cuteness',
      type: 'quantitative'
    })
  }),
  mark: 'point'
})

In [0]:
data = alt.Data(values=[{'x': 'A', 'y': 5},
                        {'x': 'B', 'y': 3},
                        {'x': 'C', 'y': 6},
                        {'x': 'D', 'y': 7},
                        {'x': 'E', 'y': 2}])
alt.Chart(data).mark_bar().encode(
    x='x:O',  # specify ordinal data
    y='y:Q',  # specify quantitative data
)

Chart({
  data: Data({
    values: [{'x': 'A', 'y': 5}, {'x': 'B', 'y': 3}, {'x': 'C', 'y': 6}, {'x': 'D', 'y': 7}, {'x': 'E', 'y': 2}]
  }),
  encoding: FacetedEncoding({
    x: X({
      field: 'x',
      type: 'ordinal'
    }),
    y: Y({
      field: 'y',
      type: 'quantitative'
    })
  }),
  mark: 'bar'
})

In [0]:
corgis = [
    {"Name":"Holly","Source":"Queen of England", 
     "Is this Real or Not Real?":"Real","Cuteness":7},
    {"Name":"Ralph","Source":"Instagram",
     "Is this Real or Not Real?":"Real","Cuteness":7},
    {"Name":"Molly (\"Thing of Evil\")","Source":"Stephen King",
     "Is this Real or Not Real?":"Real","Cuteness":9},
    {"Name":"Ein","Source":"Cowboy Bebop",
     "Is this Real or Not Real?":"Not Real","Cuteness":11},
    {"Name":"Loki","Source":"Instagram",
     "Is this Real or Not Real?":"Real","Cuteness":8},
    {"Name":"Rufus","Source":"Amazon.com",
     "Is this Real or Not Real?":"Real","Cuteness":9},
    {"Name":"Sutter Brown","Source":"Jerry Brown (gov of Cali)",
     "Is this Real or Not Real?":"Real","Cuteness":8},
    {"Name":"Hope Corgi","Source":"DC Comics",
     "Is this Real or Not Real?":"Not Real","Cuteness":11}]

corgi_alt_data = alt.Data(values = corgis)

alt.Chart(corgi_alt_data).mark_bar().encode(
    x = 'Name:N',
    y = 'Cuteness:Q',
    color = alt.Color('Name:N', legend=None)
)

Chart({
  data: Data({
    values: [{'Name': 'Holly', 'Source': 'Queen of England', 'Is this Real or Not Real?': 'Real', 'Cuteness': 7}, {'Name': 'Ralph', 'Source': 'Instagram', 'Is this Real or Not Real?': 'Real', 'Cuteness': 7}, {'Name': 'Molly ("Thing of Evil")', 'Source': 'Stephen King', 'Is this Real or Not Real?': 'Real', 'Cuteness': 9}, {'Name': 'Ein', 'Source': 'Cowboy Bebop', 'Is this Real or Not Real?': 'Not Real', 'Cuteness': 11}, {'Name': 'Loki', 'Source': 'Instagram', 'Is this Real or Not Real?': 'Real', 'Cuteness': 8}, {'Name': 'Rufus', 'Source': 'Amazon.com', 'Is this Real or Not Real?': 'Real', 'Cuteness': 9}, {'Name': 'Sutter Brown', 'Source': 'Jerry Brown (gov of Cali)', 'Is this Real or Not Real?': 'Real', 'Cuteness': 8}, {'Name': 'Hope Corgi', 'Source': 'DC Comics', 'Is this Real or Not Real?': 'Not Real', 'Cuteness': 11}]
  }),
  encoding: FacetedEncoding({
    color: Color({
      field: 'Name',
      legend: None,
      type: 'nominal'
    }),
    x: X({
      field: 'Name',
      type: 'nominal'
    }),
    y: Y({
      field: 'Cuteness',
      type: 'quantitative'
    })
  }),
  mark: 'bar'
})

In [0]:
corgi_from_json = alt.Data.from_json(corgis)

corgi_from_json

TypeError: ignored

In [60]:
import altair as alt
import pandas as pd
data = pd.DataFrame({'x': [1, 2, 3], 'y': [2, 1, 2]})

chart = alt.Chart(data).mark_line().encode(
     x='x',
     y='y'
)

from pprint import pprint
pprint(chart.to_dict())

chart

{'$schema': 'https://vega.github.io/schema/vega-lite/v3.3.0.json',
 'config': {'mark': {'tooltip': None}, 'view': {'height': 300, 'width': 400}},
 'data': {'name': 'data-2b857397a15d9ff67a8419d09d5c73c6'},
 'datasets': {'data-2b857397a15d9ff67a8419d09d5c73c6': [{'x': 1, 'y': 2},
                                                        {'x': 2, 'y': 1},
                                                        {'x': 3, 'y': 2}]},
 'encoding': {'x': {'field': 'x', 'type': 'quantitative'},
              'y': {'field': 'y', 'type': 'quantitative'}},
 'mark': 'line'}


Chart({
  data:    x  y
  0  1  2
  1  2  1
  2  3  2,
  encoding: FacetedEncoding({
    x: X({
      field: 'x',
      type: 'quantitative'
    }),
    y: Y({
      field: 'y',
      type: 'quantitative'
    })
  }),
  mark: 'line'
})

In [0]:
alt.data_transformers.enable('json')

url = 'data.json'
data.to_json(url, orient='records')

chart = alt.Chart(url).mark_line().encode(
    x='x:Q',
    y='y:Q'
)
pprint(chart.to_dict())

chart

{'$schema': 'https://vega.github.io/schema/vega-lite/v3.3.0.json',
 'config': {'mark': {'tooltip': None}, 'view': {'height': 300, 'width': 400}},
 'data': {'url': 'data.json'},
 'encoding': {'x': {'field': 'x', 'type': 'quantitative'},
              'y': {'field': 'y', 'type': 'quantitative'}},
 'mark': 'line'}


Chart({
  data: 'data.json',
  encoding: FacetedEncoding({
    x: X({
      field: 'x',
      type: 'quantitative'
    }),
    y: Y({
      field: 'y',
      type: 'quantitative'
    })
  }),
  mark: 'line'
})

In [0]:
c_data = pd.read_csv('corgi_named.csv', index_col = None)
c_data

,Name,Source,Is this Real or Not Real?,Cuteness
0,Holly,Queen of England,Real,7
1,Ralph,Instagram,Real,7
2,"Molly (""Thing of Evil"")",Stephen King,Real,9
3,Ein,Cowboy Bebop,Not Real,11
4,Loki,Instagram,Real,8
5,Rufus,Amazon.com,Real,9
6,Sutter Brown,Jerry Brown (gov of Cali),Real,8
7,Hope Corgi,DC Comics,Not Real,11


In [0]:
c_chart = alt.Chart(corgi_data).mark_bar().encode(
    x = 'Name',
    y = 'Cuteness'
)

pprint(c_chart.to_dict())

{'$schema': 'https://vega.github.io/schema/vega-lite/v3.3.0.json',
 'config': {'mark': {'tooltip': None}, 'view': {'height': 300, 'width': 400}},
 'data': {'name': 'data-934fbd9a05fd9d3c7b4838d1b504f158'},
 'datasets': {'data-934fbd9a05fd9d3c7b4838d1b504f158': [{'Cuteness': 7,
                                                         'Is this Real or Not Real?': 'Real',
                                                         'Name': 'Holly',
                                                         'Source': 'Queen of '
                                                                   'England'},
                                                        {'Cuteness': 7,
                                                         'Is this Real or Not Real?': 'Real',
                                                         'Name': 'Ralph',
                                                         'Source': 'Instagram'},
                                                        {'Cuteness': 9,
   

In [0]:
url = 'c_data.json'
c_data.to_json(url, orient = 'records')

c_chart = alt.Chart(url).mark_bar().encode(
    x = 'Name:N',
    y = 'Cuteness:Q'
)

pprint(c_chart.to_dict())

{'$schema': 'https://vega.github.io/schema/vega-lite/v3.3.0.json',
 'config': {'mark': {'tooltip': None}, 'view': {'height': 300, 'width': 400}},
 'data': {'url': 'c_data.json'},
 'encoding': {'x': {'field': 'Name', 'type': 'nominal'},
              'y': {'field': 'Cuteness', 'type': 'quantitative'}},
 'mark': 'bar'}


test basic bar chart, large data, not joined

In [0]:
simple_bar_graph = make_graph(algSource, 'algorithm', 'runtime', 'bar')

In [16]:
simple_bar_graph

MaxRowsError: ignored

Chart({
  data:                                              instance_id  ...  runstatus
  0      FolioSuite/ASP-Comp-2007-Lparse/SLparse/15-Puz...  ...         ok
  1      FolioSuite/ASP-Comp-2007-Lparse/SLparse/Blocke...  ...         ok
  2      FolioSuite/ASP-Comp-2007-Lparse/SLparse/Blocke...  ...         ok
  3      FolioSuite/ASP-Comp-2007-Lparse/SLparse/Blocke...  ...         ok
  4      FolioSuite/ASP-Comp-2007-Lparse/SLparse/Blocke...  ...         ok
  5      FolioSuite/ASP-Comp-2007-Lparse/SLparse/Bounde...  ...         ok
  6      FolioSuite/ASP-Comp-2007-Lparse/SLparse/Bounde...  ...         ok
  7      FolioSuite/ASP-Comp-2007-Lparse/SLparse/Bounde...  ...         ok
  8      FolioSuite/ASP-Comp-2007-Lparse/SLparse/Bounde...  ...         ok
  9      FolioSuite/ASP-Comp-2007-Lparse/SLparse/Bounde...  ...         ok
  10     FolioSuite/ASP-Comp-2007-Lparse/SLparse/CarSeq...  ...    timeout
  11     FolioSuite/ASP-Comp-2007-Lparse/SLparse/CarSeq...  ...    timeout
  12     Fo

In [18]:
algSource.head()

,instance_id,repetition,algorithm,runtime,runstatus
0,Instance_0,1.0,algorithm_0,3600.000,memout
1,Instance_0,1.0,algorithm_1,3600.000,memout
2,Instance_0,1.0,id_algorithm_0,311.863,ok
3,Instance_0,1.0,id_algorithm_1,3600.000,timeout
4,Instance_1,1.0,algorithm_0,3600.000,memout


In [19]:
alt.Chart(algSource).mark_circle().encode(
    x = 'runtime:Q',
    y = 'runtime_y:Q'
).transform_lookup(
    lookup='comparison',
    from_=alt.LookupData(data=algSource, key = 'instance_id',
                        fields = as('runtime'))#???
)

SyntaxError: ignored

practice joinable tables (based on characters from The Dresden Chronicles by Jim Butcher)

In [0]:
friend = {
    'name': ['karrin murphy', 'thomas raith', 'susan rodriguez', 
             'molly carpenter', 'major general minimus toot-toot', 'lady lily',
             'elaine mallory'],
    
    'monster_type': ['human', 'white court vampire', 'red court vampire',
                     'warlock', 'fae', 'fae', 'warlock'],
    
    'life_status': ['living', 'living', 'dead', 'living', 'living', 'dead', 
                    'living'],
    
    'favoritism_points': [10, 8, 7, 5, 3, 2, 1]
}

monster = {
    'type': ['human', 'white court vampire', 'red court vampire', 
                     'warlock', 'fae'],
    
    'danger_points': [2, 6, 5, 5, 10]
}

friends = pd.DataFrame(friend)
monsters = pd.DataFrame(monster)

In [29]:
alt.Chart(friends).mark_bar().encode(
    x = 'name:N',
    y = 'favoritism_points:Q'
)

Chart({
  data:                               name  ... favoritism_points
  0                    karrin murphy  ...                10
  1                     thomas raith  ...                 8
  2                  susan rodriguez  ...                 7
  3                  molly carpenter  ...                 5
  4  major general minimus toot-toot  ...                 3
  5                        lady lily  ...                 2
  6                   elaine mallory  ...                 1
  
  [7 rows x 4 columns],
  encoding: FacetedEncoding({
    x: X({
      field: 'name',
      type: 'nominal'
    }),
    y: Y({
      field: 'favoritism_points',
      type: 'quantitative'
    })
  }),
  mark: 'bar'
})

In [64]:
fave_vs_danger = alt.Chart(friends).mark_circle(size=100).encode(
    x = 'danger:Q',
    y = 'favoritism_points:Q',
    tooltip = ['name', 'favoritism_points', 'monster_type', 'danger:Q'],
    color = alt.Color('monster_type')
).transform_lookup(
    lookup = 'monster_type',
    from_ = alt.LookupData(data=monsters, key = 'type',
                           fields = ['danger_points']),
    as_ = ['danger']
)

fave_vs_danger

Chart({
  data:                               name  ... favoritism_points
  0                    karrin murphy  ...                10
  1                     thomas raith  ...                 8
  2                  susan rodriguez  ...                 7
  3                  molly carpenter  ...                 5
  4  major general minimus toot-toot  ...                 3
  5                        lady lily  ...                 2
  6                   elaine mallory  ...                 1
  
  [7 rows x 4 columns],
  encoding: FacetedEncoding({
    color: Color({
      field: 'monster_type',
      type: 'nominal'
    }),
    tooltip: [Tooltip({
      field: 'name',
      type: 'nominal'
    }), Tooltip({
      field: 'favoritism_points',
      type: 'quantitative'
    }), Tooltip({
      field: 'monster_type',
      type: 'nominal'
    }), Tooltip({
      field: 'danger',
      type: 'quantitative'
    })],
    x: X({
      field: 'danger',
      type: 'quantitative'
    }),
    y: Y({
      field: 'favoritism_points',
      type: 'quantitative'
    })
  }),
  mark: MarkDef({
    size: 100,
    type: 'circle'
  }),
  transform: [LookupTransform({
    as: ['danger'],
    from: LookupData({
      data:                   type  danger_points
      0                human              2
      1  white court vampire              6
      2    red court vampire              5
      3              warlock              5
      4                  fae             10,
      fields: ['danger_points'],
      key: 'type'
    }),
    lookup: 'monster_type'
  })]
})

In [41]:
from vega_datasets import data
people = data.lookup_people()
groups = data.lookup_groups()

,group,person
0,1,Alan
1,1,George
2,1,Fred
3,2,Steve
4,2,Nick


In [42]:
groups.head()

,group,person
0,1,Alan
1,1,George
2,1,Fred
3,2,Steve
4,2,Nick


In [43]:
people.head()

,name,age,height
0,Alan,25,180
1,George,32,174
2,Fred,39,182
3,Steve,42,161
4,Nick,23,180


In [0]:
fave_vs_danger.save('fave_vs_danger.html')

In [0]:
fave_int = fave_vs_danger.interactive()
fave_int.save('fave_int.html')

In [61]:
pprint(fave_int.to_dict())

{'$schema': 'https://vega.github.io/schema/vega-lite/v3.3.0.json',
 'config': {'mark': {'tooltip': None}, 'view': {'height': 300, 'width': 400}},
 'data': {'name': 'data-cca94c3c58524e39a67bf5f8b150fdae'},
 'datasets': {'data-47489b3fdc0ce3ba2738c6de13bc98bd': [{'danger_points': 2,
                                                         'type': 'human'},
                                                        {'danger_points': 6,
                                                         'type': 'white court '
                                                                 'vampire'},
                                                        {'danger_points': 5,
                                                         'type': 'red court '
                                                                 'vampire'},
                                                        {'danger_points': 5,
                                                         'type': 'warlock'},
                      

In [0]:
alt.Chart(friends).mark_circle(size=100).encode(
    x = 'favoritism_points:Q',
    y = 'favoritsm_y:Q',
    tooltip = ['name', 'monster_type']
).transform_lookup(
    lookup=''
)